In [6]:
import numpy as np
import pandas as pd
from datetime import date, datetime, timedelta 
import re    
import json    
import time
import sys
sys.path.append('../')
from analysts_tools.growth import *
from analysts_tools.frucap import *
from analysts_tools.Clusters_querys import *
from analysts_tools.whitelist_functions import *

from procurement_lib import send_slack_notification
from procurement_lib import DataWarehouse, GoogleSheet
from analystcommunity import frubana_logger as logging
import logging as pylogging
import warnings
import pytz
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk import ngrams
from nltk.stem import WordNetLemmatizer
from nltk import wordnet
from fuzzywuzzy import process
from fuzzywuzzy import fuzz
import nltk
from collections import Counter
from itertools import chain
import itertools
from itertools import repeat
import math
from procurement_lib import redash

dw = DataWarehouse()
#warnings.filterwarnings("ignore")

from procurement_lib import redash
from analysts_tools.redash_methods import *

In [7]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros
todays_date = '2022-07-04'  
customer_date_desde = '2022-06-20' #3 months

In [8]:
query = """
    WITH 
    count_fulfillments_pasados AS (
        SELECT 
            count(distinct boi.order_item_id) as count_items,
            bfg.order_id
        FROM postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"              bfg
        LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order"                     bo      ON bo.order_id = bfg.order_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"          ffg     ON bfg.fulfillment_group_id = ffg.fulfillment_group_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"         bfo     ON bfg.fulfillment_group_id = bfo.fulfillment_group_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment"             bop     ON bop.order_id = bo.order_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_order"                      fo      ON fo.order_id = bo.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON bo.site_disc = s.site_id
        LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"                fot     ON fot.fb_order_type_id=fo.fb_order_type_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"   bfgi    ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON bfgi.order_item_id= boi.order_item_id
        
        WHERE      
            ffg.close_date >= '{desde}' AND ffg.close_date <= '{hasta}'
            AND bo.order_status='SUBMITTED'
            AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
            AND fo.fb_order_status_id IN (1,6,7,8)
            AND bop.archived = 'N'
            AND (fot.name IS NULL OR fot.name <> 'REFUND')
        GROUP BY 2
    ),

    ful as (

        SELECT DISTINCT
            bo.order_id,
            ffg.close_date
            
        FROM postgres_broadleaf_federate."broadleaf.blc_order"                      bo
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"    bfg ON bfg.order_id = bo.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order"    bfo ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"     ffg ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                 fo  ON fo.order_id = bo.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment"        bop ON bop.order_id = bo.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                 s   ON s.site_id = bo.site_disc
        LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"            fot ON fot.fb_order_type_id=fo.fb_order_type_id
        LEFT JOIN  postgres_broadleaf_federate."broadleaf.blc_order_adjustment"     ba  ON ba.order_id = bo.order_id
        
        WHERE fo.fb_order_status_id IN (1,6,7,8)
            AND ffg.close_date >= '{desde}' AND ffg.close_date <= '{hasta}'
            AND bo.order_status = 'SUBMITTED'
            AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
            AND bop.archived = 'N'
            AND (fot.name IS NULL OR fot.name <> 'REFUND')
            
        GROUP BY bo.order_id, ffg.close_date
    )--,

    --all_data AS (

    SELECT DISTINCT
        s.site_identifier_value as region_code,
        bcat2.name as cat,
        bcat.name as subcat,
        bs.sku_id,
        boi.name AS product,
        bo.customer_id,
        --count(distinct bo.customer_id) as penetracion,
        CASE 
            WHEN s.site_identifier_value = 'CMX' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
            WHEN s.site_identifier_value = 'GDL' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
            WHEN s.site_identifier_value = 'PBC' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
            WHEN s.site_identifier_value = 'MTY' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/19.19)
            WHEN s.site_identifier_value = 'SPO' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
            WHEN s.site_identifier_value = 'BHZ' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
            WHEN s.site_identifier_value = 'CWB' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
            WHEN s.site_identifier_value = 'VCP' then ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3.88)
        ELSE ((SUM(boi.quantity * foi.step_unit * boi.sale_price) + SUM((coalesce(bo.total_shipping,0) / cf.count_items)) + coalesce(SUM(foi.total_tax_iva),0))*1.0/3000) END AS net_gmv_usd

    FROM 
        postgres_broadleaf_federate."broadleaf.blc_fulfillment_group_item"                  bfgi
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group"            bfg     ON bfgi.fulfillment_group_id = bfg.fulfillment_group_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group"             ffg     ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order"                        bo      ON bo.order_id = bfg.order_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                         s       ON bo.site_disc = s.site_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"                   boi     ON bfgi.order_item_id= boi.order_item_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                    foi     ON boi.order_item_id= foi.order_item_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_owner"                         fow     ON fow.owner_id = ffg.owner_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                         fo      ON fo.order_id = bo.order_id
        INNER JOIN ful                                                                              ON ful.order_id = bo.order_id
        LEFT JOIN count_fulfillments_pasados                                                cf      ON bfg.order_id = cf.order_id
        LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type"                    fot     ON fot.fb_order_type_id=fo.fb_order_type_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"          bdoi    ON bdoi.order_item_id = boi.order_item_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                          bs      ON bs.sku_id = bdoi.sku_id
        INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                      bp      ON bs.addl_product_id = bp.product_id
        left join postgres_broadleaf_federate."broadleaf.blc_category_xref"                 bcx     ON bcx.sub_category_id = bp.default_category_id and bcx.archived='N' and bcx.sndbx_tier is null and bcx.default_reference = 'true'
        left join postgres_broadleaf_federate."broadleaf.blc_category"                      bcat    ON bp.default_category_id = bcat.category_id
        left join postgres_broadleaf_federate."broadleaf.blc_category"                      bcat2   ON bcx.category_id = bcat2.category_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"          boipd   ON boipd.order_item_id=boi.order_item_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"            boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id
        LEFT JOIN postgres_broadleaf_federate."broadleaf.fb_customer"              fc         ON fc.customer_id = bo.customer_id

    WHERE 
        ffg.close_date >= '{desde}' AND ffg.close_date <= '{hasta}'
        AND bo.order_status='SUBMITTED'
        AND fo.fb_order_status_id IN (1,6,7,8)
        AND (fot.name IS NULL OR fot.name <> 'REFUND')
        AND bcat2.name in ('Abarrotes','Aseo e Higiene','Bebidas','Congelados','Desechables','Lácteos & Huevos','Carne, Pollo & Pescados', 'Frutas & Verduras', 'Bebidas','Mercearia','Limpeza e Higiene','Laticínios e Ovos','Frutas e Verduras','Congelados','Descartáveis','Carnes, Aves e Peixes')
        AND fc.business_type_id IN (1)
        
    GROUP BY 1,2,3,4,5,6
""".format(desde=customer_date_desde, hasta=todays_date)
data_ventas = read_connection_data_warehouse.runQuery(query)

In [9]:
data_ventas

,region_code,cat,subcat,sku_id,product,customer_id,net_gmv_usd
0,SPO,Frutas e Verduras,Legumes,-314710,Pimentão Vermelho Padrão - Kg,26954116,6.170103092783505154639100
1,SPO,Mercearia,Farinhas e Misturas,108304,Farinha de Trigo Sol 1Kg 1Kg - Mais de 10 uni...,26954116,36.185567010309278350515400
2,SPO,Frutas e Verduras,Verduras,75087,Salsinha Maço Padrão - Unidade,35223933,0.688144329896907216494800
3,SPO,Laticínios e Ovos,Ovos,247540,Ovos Brancos Grandes Katayama Cinta com 5 dúz...,35223933,64.569587628865979381443200
4,SPO,Frutas e Verduras,Verduras,112100,Alface Crespa Padrão - Unidade Atacado,44367560,22.329896907216494845360800
...,...,...,...,...,...,...,...
694146,MDE,Frutas & Verduras,Verduras,363445,Cilantro Patirojo Estándar - Atado,171480896,1.470000000000000000000000
694147,SPO,Mercearia,"Azeites, Óleos e Vinagres",384239,Óleo Vitaliv 900ml PET 900ml - Unidade,113598298,19.567010309278350515463900
694148,GDL,Abarrotes,Aceites & Grasas,503712,Aceite Vegetal Mixto Ave 800 ml Botella de 80...,175611540,22.127149557060969254820200
694149,SPO,Frutas e Verduras,Frutas,-314629,Limão Taiti Padrão - Kg,181711981,0.695876288659793814432900


In [10]:
Query_SKU=get_fresh_query_result("https://internal-redash.federate.frubana.com/",97797,'9FFFsAzt85oV0jqIpdPolKrUJvjKTapU1Q89d1FD',{},2)

200


In [ ]:
Query_SKU.head()

,category,card_frida,sku_id,subcategory,card_name,ciudad
0,Frutas & Verduras,-200011,-303396,Verduras,Cebolla Cabezona Blanca Limpia Pequeña Kg,BAQ
1,Frutas & Verduras,-200891,-303393,Verduras,Laurel Estandar Atado,BAQ
2,Frutas & Verduras,-200773,-303390,Verduras,Tomate Cherry Estandar Bandeja,BAQ
3,Frutas & Verduras,-200027,-303369,Frutas,Limón Pajarito Estándar Kg - 🤑 (Insuperable),BAQ
4,"Carne, Pollo & Pescados",-201051,-303231,Embutidos,Chorizo Parrilla Paquete 500g Paquete,BAQ


In [ ]:
Query_MS=get_fresh_query_result("https://internal-redash.federate.frubana.com/",82561,'9FFFsAzt85oV0jqIpdPolKrUJvjKTapU1Q89d1FD',{},20)

200


In [31]:
Query_MS.head()

,city,microsegment,customer_id,microsegment_id
0,CMX,Fonda c/desayuno,-269932,13
1,CMX,Fonda c/desayuno,-269919,13
2,CMX,Fonda s/desayuno,-269876,14
3,CMX,Fonda c/desayuno,-269858,13
4,CMX,Juguería/Comida saludable,-269813,17


In [32]:
regiones=data_ventas[['customer_id','region_code']].drop_duplicates()

In [33]:
regiones

,customer_id,region_code
0,54045002,SPO
1,102319612,CMX
2,142294002,BAQ
3,45165430,BOG
4,163910249,SPO
...,...,...
2551850,54787682,GDL
2555698,96164007,CMX
2557474,104068996,BHZ
2557701,128427230,SPO


In [34]:
Query_MS_data=pd.merge(Query_MS,regiones,on='customer_id')

In [35]:
Query_MS_data

,city,microsegment,customer_id,microsegment_id,region_code
0,CMX,Fonda c/desayuno,-269919,13,CMX
1,CMX,Juguería/Comida saludable,-269813,17,CMX
2,CMX,Juguería/Comida saludable,-269782,17,CMX
3,CMX,Fonda c/desayuno,-269757,13,CMX
4,CMX,Fonda c/desayuno,-269745,13,CMX
...,...,...,...,...,...
56639,CMX,Comida rápida,178657765,1,CMX
56640,CMX,Taquería en calle,178680896,15,CMX
56641,CMX,Comida rápida,178696918,1,CMX
56642,CMX,Fonda c/desayuno,178808760,13,CMX


In [36]:
Query_MS_correct_MS=MS_corregidos_2(Query_MS_data)

In [37]:
Query_MS_correct_MS

,city,microsegment,customer_id,microsegment_id,region_code
0,CMX,Fonda c/desayuno,-269919,13,CMX
1,CMX,Juguería/Comida saludable,-269813,17,CMX
2,CMX,Juguería/Comida saludable,-269782,17,CMX
3,CMX,Fonda c/desayuno,-269757,13,CMX
4,CMX,Fonda c/desayuno,-269745,13,CMX
...,...,...,...,...,...
56639,CMX,Comida rápida,178657765,1,CMX
56640,CMX,Taquería en calle,178680896,15,CMX
56641,CMX,Comida rápida,178696918,1,CMX
56642,CMX,Fonda c/desayuno,178808760,13,CMX


In [38]:
Query_SKU.head()

,category,card_frida,sku_id,subcategory,card_name,ciudad
0,Frutas & Verduras,-200011,-303396,Verduras,Cebolla Cabezona Blanca Limpia Pequeña Kg,BAQ
1,Frutas & Verduras,-200891,-303393,Verduras,Laurel Estandar Atado,BAQ
2,Frutas & Verduras,-200773,-303390,Verduras,Tomate Cherry Estandar Bandeja,BAQ
3,Frutas & Verduras,-200027,-303369,Frutas,Limón Pajarito Estándar Kg - 🤑 (Insuperable),BAQ
4,"Carne, Pollo & Pescados",-201051,-303231,Embutidos,Chorizo Parrilla Paquete 500g Paquete,BAQ


In [11]:
df_prev=pd.merge(data_ventas,Query_SKU, on = 'sku_id')
#df=pd.merge(df_prev,Query_MS_correct_MS,how = 'left', on = 'customer_id')

In [20]:
df_top = df_prev.groupby(by=['region_code', 'cat', 'subcat', 'card_frida','card_name']).agg({'net_gmv_usd':np.sum, 'customer_id': 'nunique'}).reset_index()

In [25]:
df_top.sort_values(['customer_id'],ascending = False)

,region_code,cat,subcat,card_frida,card_name,net_gmv_usd,customer_id
8449,SPO,Frutas e Verduras,Frutas,-202919,Limão Taiti Padrão Kg,13017.96392446412155326609600,5922
8494,SPO,Frutas e Verduras,Legumes,-202941,Cebola Padrão Kg,27202.10069540997376048898220,4721
8538,SPO,Frutas e Verduras,Tubérculos,-202908,Batata Lavada Padrão Padrão Kg,36846.64852366873743465596790,4499
8488,SPO,Frutas e Verduras,Legumes,-202950,Tomate Italiano . Kg,32726.32792368082695553868360,4419
8983,SPO,Mercearia,"Azeites, Óleos e Vinagres",293209,Óleo Vitaliv 900ml PET 900ml Mais de 20 unidades,376058.5567010309278350514440,3830
...,...,...,...,...,...,...,...
6187,GDL,Congelados,Alimentos Congelados,241362,Papas Camote Corte Recto 3/8 x 6/16 - Simplot ...,15.972902553413236060448100,1
2696,BOG,Desechables,Complementos,271673,Fosforos Bengala madera x 2 cajas Paquete x 2 ...,7.228853333333333333333333,1
2695,BOG,Desechables,Complementos,271673,Fosforos Bengala madera x 2 cajas Paquete x 2 ...,3.860360000000000000000000,1
8337,SPO,Descartáveis,Embalagens,251517,Marmitex de alumínio Wyda 850ml Pacote com 100...,12.615979381443298969072100,1


In [33]:
for city in df_top.region_code.unique():
     s = df_top[df_top['region_code']==city].sort_values(['customer_id'],ascending = False).head(10)
     s.to_csv(f'Analisis_{city}.csv', index = False)
    

In [24]:
a

,region_code,cat,subcat,card_frida,card_name,net_gmv_usd,customer_id
0,BAQ,Abarrotes,Aceites & Grasas,-200951,Aceite Riquísimo 1LT Botella x 1000ml Unidad,392.475769999999999999999986,27
1,BAQ,Abarrotes,Aceites & Grasas,62989,Aceite sólido Bucaro 15kg Caja de 15kg Unidad,1395.330533333333333333333330,9
2,BAQ,Abarrotes,Aceites & Grasas,180605,Aceite Naoli 20LT Bidón por 20 litros Unidad,47106.47432999999999999999968,347
3,BAQ,Abarrotes,Aceites & Grasas,180812,Aceite Naoli 3LT Botella por 3 litros Unidad,49950.38573142857142857142904,1420
4,BAQ,Abarrotes,Aceites & Grasas,250059,Margarina Andiponque x 15 Kg Caja x 15 kg Desd...,1507.350119999999999999999992,26
...,...,...,...,...,...,...,...
9898,VCP,Mercearia,Temperos,356515,Mais Sabor Carne Kitano 60g Pacote de 60g A pa...,35.234536082474226804123100,14
9899,VCP,Mercearia,Temperos,356516,Mais Sabor Feijão Kitano 60g Pacote de 60g A p...,15.819587628865979381443100,5
9900,VCP,Mercearia,Temperos,356518,Orégano Kitano 200g Pacote de 200g A partir de...,28.432989690721649484536000,7
9901,VCP,Mercearia,Temperos,356519,Pimenta Preta em Pó Kitano 50g Pacote de 50g A...,6.559278350515463917525700,2


In [14]:
df_prev.to_excel(f'Analisis.xlsx', index = False)

KeyboardInterrupt: 

In [16]:
df_prev.to_csv(f'Analisis.csv', index = False)

In [40]:
df=df.rename(columns={'region_code_x':'region_code'})

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2545431 entries, 0 to 2545430
Data columns (total 16 columns):
 #   Column           Dtype  
---  ------           -----  
 0   region_code      object 
 1   cat              object 
 2   subcat           object 
 3   sku_id           int64  
 4   product          object 
 5   customer_id      int64  
 6   net_gmv_usd      object 
 7   category         object 
 8   card_frida       int64  
 9   subcategory      object 
 10  card_name        object 
 11  ciudad           object 
 12  city             object 
 13  microsegment     object 
 14  microsegment_id  float64
 15  region_code_y    object 
dtypes: float64(1), int64(3), object(12)
memory usage: 330.1+ MB


In [42]:
df.head(10)

,region_code,cat,subcat,sku_id,product,customer_id,net_gmv_usd,category,card_frida,subcategory,card_name,ciudad,city,microsegment,microsegment_id,region_code_y
0,SPO,"Carnes, Aves e Peixes",Peixes e Frutos do Mar,232252,Filé de Tilápia sem Pele - Rakki Congelado - ...,54045002,54.397938144329896907216400,"Carnes, Aves e Peixes",188112,Peixes e Frutos do Mar,Filé de Tilápia sem Pele sem Espinha - Rakki C...,SPO,NaN,NaN,NaN,NaN
1,SPO,"Carnes, Aves e Peixes",Peixes e Frutos do Mar,232252,Filé de Tilápia sem Pele - Rakki Congelado - ...,54786620,38.855670103092783505154600,"Carnes, Aves e Peixes",188112,Peixes e Frutos do Mar,Filé de Tilápia sem Pele sem Espinha - Rakki C...,SPO,NaN,NaN,NaN,NaN
2,SPO,"Carnes, Aves e Peixes",Peixes e Frutos do Mar,232252,Filé de Tilápia sem Pele - Rakki Congelado - ...,38186291,38.855670103092783505154600,"Carnes, Aves e Peixes",188112,Peixes e Frutos do Mar,Filé de Tilápia sem Pele sem Espinha - Rakki C...,SPO,NaN,NaN,NaN,NaN
3,SPO,"Carnes, Aves e Peixes",Peixes e Frutos do Mar,232252,Filé de Tilápia sem Pele - Rakki Congelado - ...,35827347,38.855670103092783505154600,"Carnes, Aves e Peixes",188112,Peixes e Frutos do Mar,Filé de Tilápia sem Pele sem Espinha - Rakki C...,SPO,NaN,NaN,NaN,NaN
4,SPO,"Carnes, Aves e Peixes",Peixes e Frutos do Mar,232252,Filé de Tilápia sem Pele - Rakki Congelado - ...,100136195,38.855670103092783505154600,"Carnes, Aves e Peixes",188112,Peixes e Frutos do Mar,Filé de Tilápia sem Pele sem Espinha - Rakki C...,SPO,SPO,Comida brasileira y saudavel,23.0,SPO
5,SPO,"Carnes, Aves e Peixes",Peixes e Frutos do Mar,232252,Filé de Tilápia sem Pele - Rakki Congelado - ...,63454440,38.855670103092783505154600,"Carnes, Aves e Peixes",188112,Peixes e Frutos do Mar,Filé de Tilápia sem Pele sem Espinha - Rakki C...,SPO,SPO,Prato do dia c/café da manhã,19.0,SPO
6,SPO,"Carnes, Aves e Peixes",Peixes e Frutos do Mar,232252,Filé de Tilápia sem Pele - Rakki Congelado - ...,128943200,38.855670103092783505154600,"Carnes, Aves e Peixes",188112,Peixes e Frutos do Mar,Filé de Tilápia sem Pele sem Espinha - Rakki C...,SPO,NaN,NaN,NaN,NaN
7,SPO,"Carnes, Aves e Peixes",Peixes e Frutos do Mar,232252,Filé de Tilápia sem Pele - Rakki Congelado - ...,35203692,38.855670103092783505154600,"Carnes, Aves e Peixes",188112,Peixes e Frutos do Mar,Filé de Tilápia sem Pele sem Espinha - Rakki C...,SPO,NaN,NaN,NaN,NaN
8,SPO,"Carnes, Aves e Peixes",Peixes e Frutos do Mar,232252,Filé de Tilápia sem Pele - Rakki Congelado - ...,52801530,38.855670103092783505154600,"Carnes, Aves e Peixes",188112,Peixes e Frutos do Mar,Filé de Tilápia sem Pele sem Espinha - Rakki C...,SPO,NaN,NaN,NaN,NaN
9,SPO,"Carnes, Aves e Peixes",Peixes e Frutos do Mar,232252,Filé de Tilápia sem Pele - Rakki Congelado - ...,19767992,38.855670103092783505154600,"Carnes, Aves e Peixes",188112,Peixes e Frutos do Mar,Filé de Tilápia sem Pele sem Espinha - Rakki C...,SPO,SPO,Prato do dia c/café da manhã,19.0,SPO


In [43]:
def analizar_top_productos(data_ventas):
    """
    Está función recibe un dataframe con información de ventas (sale de la función data_ventas_query) y analiza, por sku, cuales son los 
    top productos.
    los criterios bajo el cual la función define top productos son 2: (1) penetración del sku_id (2) gmv_usd que mueve el sku_id.
    
    Argumentos:
    data_ventas: data de ventas que entrega la función data_ventas_query.
    
    Resultado:
    top_productos_df: dataframe ordenada con los top productos
    """
    def group_top_products(x):
        resultado = {}
        resultado["card_name"] = x["card_name"].unique()[0]
        resultado["category"] = x["category"].unique()[0]
        assert len(x["category"].unique()) == 1
        resultado["subcategory"] = x["subcategory"].unique()[0]
        assert len(x["subcategory"].unique()) == 1
        resultado["microsegment"] = x["microsegment"].unique()[0]
        assert len(x["microsegment"].unique()) == 1
        resultado["penetration"] = x["customer_id"].nunique()
        resultado["gmv_usd"] = x["net_gmv_usd"].sum()
        #assert len(x["sku_id"].unique()) == 1
        resultado["sku_id"] = '#'.join(x["sku_id"].unique())
        return pd.Series(resultado, index=list(resultado.keys()))
    
    data_ventas[["sku_id", "card_frida"]] = data_ventas[["sku_id", "card_frida"]].astype(str)

    top_productos_df = data_ventas.groupby(by=["card_frida"]).apply(group_top_products).reset_index()
    top_productos_df = top_productos_df.sort_values(by=["penetration", "gmv_usd"], ascending=False)
    num_customer_ids = data_ventas["customer_id"].nunique()
    gmv_total = data_ventas["net_gmv_usd"].sum()

    top_productos_df["penetration_porcentual"] = 100*top_productos_df["penetration"]/num_customer_ids
    top_productos_df["gmv_p_user"] = 100*top_productos_df["gmv_usd"]/num_customer_ids

    return top_productos_df

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2545431 entries, 0 to 2545430
Data columns (total 16 columns):
 #   Column           Dtype  
---  ------           -----  
 0   region_code      object 
 1   cat              object 
 2   subcat           object 
 3   sku_id           int64  
 4   product          object 
 5   customer_id      int64  
 6   net_gmv_usd      object 
 7   category         object 
 8   card_frida       int64  
 9   subcategory      object 
 10  card_name        object 
 11  ciudad           object 
 12  city             object 
 13  microsegment     object 
 14  microsegment_id  float64
 15  region_code_y    object 
dtypes: float64(1), int64(3), object(12)
memory usage: 330.1+ MB


In [45]:
cities=['BOG','BAQ','CMX','SPO']

df['microsegment']=df.apply(lambda x: x['microsegment'] if x['region_code'] in cities else 'Other',axis=1)
df['microsegment']=df['microsegment'].fillna('Other')

In [59]:
cities=['BOG','BAQ','MDE','CMX','GDL','PBC','SPO','BHZ','CWB','VCP']
DFDictCitySKU = {elem : df[df['region_code'] == elem] for elem in cities}

for i in cities:
    for j in DFDictCitySKU[i].microsegment.unique():
        filtro_ms= (DFDictCitySKU[i]['microsegment']==j)
        BASE=DFDictCitySKU[i][filtro_ms]
        DFDictCitySKU[i+'_sku_'+j]=analizar_top_productos(BASE)
        DFDictCitySKU[i+'_sku_'+j]['region_code']=i
        k=j.replace("/","_")
        #DFDictCitySKU[i+'_sku_'+j].to_csv(f'Top_skus_MS/top_{i}_skus_{k}.csv', index=False)

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/frame.py:3191: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[k1] = value[k2]


In [68]:
i='SPO'
j='Lanches'
print(DFDictCitySKU[i+'_sku_'+j]['gmv_usd'].sum())
print(DFDictCitySKU[i+'_sku_'+j]['card_frida'].nunique())

DFDictCitySKU[i+'_sku_'+j].groupby(by = ['region_code','microsegment', 'category', 'subcategory']).agg({'penetration':np.sum,'gmv_usd':np.sum}).reset_index().info()

2860872.100399682522181001145
864
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   region_code   45 non-null     object
 1   microsegment  45 non-null     object
 2   category      45 non-null     object
 3   subcategory   45 non-null     object
 4   penetration   45 non-null     int64 
 5   gmv_usd       45 non-null     object
dtypes: int64(1), object(5)
memory usage: 2.2+ KB


In [75]:
df_ms = pd.DataFrame(columns = ['region_code', 'microsegment','category','subcategory','penetration','gmv_usd'])
for i in cities:
    for j in DFDictCitySKU[i].microsegment.unique():
        df2 = DFDictCitySKU[i+'_sku_'+j].groupby(by = ['region_code','microsegment', 'category', 'subcategory']).agg({'penetration':np.sum,'gmv_usd':np.sum}).reset_index()
        df_ms = pd.concat([df_ms, df2], ignore_index=True)

In [79]:
df_ms.to_excel(f'Analisis.xlsx', index = False)

In [1]:
analystcommunity??

Object `analystcommunity` not found.
